In [34]:
from itertools import chain
from collections import Counter
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import KBinsDiscretizer, StandardScaler
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import sys

%matplotlib inline 

In [ ]:
## Introduction to Linear Regression 

In [14]:
import pandas as pd
url = 'https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv?accessType=DOWNLOAD'
data_raw = pd.read_csv(url)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
# ## download for MAC
# file_path = 'nyc_caraccidents.csv'
# path = Path(file_path)
# response = requests.get(url)

# # write contents of the download 
# path.write_bytes(response.content)

# data = pd.read_csv(file_path)

,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,08/26/2018,14:00,NaN,NaN,40.725975,-73.75750,POINT (-73.7575 40.725975),GRAND CENTRAL PKWY,NaN,NaN,...,Unspecified,NaN,NaN,NaN,3968078,Sedan,Sedan,NaN,NaN,NaN
1,09/14/2018,8:14,BROOKLYN,11210,40.633320,-73.95394,POINT (-73.95394 40.63332),GLENWOOD ROAD,EAST 24 STREET,NaN,...,Unspecified,NaN,NaN,NaN,3980557,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
2,09/04/2018,15:30,BRONX,10473,40.823826,-73.86163,POINT (-73.86163 40.823826),STORY AVENUE,LELAND AVENUE,NaN,...,Unspecified,Unspecified,NaN,NaN,3974356,Sedan,Sedan,Taxi,NaN,NaN
3,09/16/2018,17:00,BROOKLYN,11231,40.672226,-74.01101,POINT (-74.01101 40.672226),BEARD STREET,OTSEGO STREET,NaN,...,Unspecified,Unspecified,NaN,NaN,3982548,Sedan,Sedan,Sedan,NaN,NaN
4,08/27/2018,2:00,NaN,NaN,NaN,NaN,NaN,MEEKER AVENUE,VANDERVORT AVENUE,NaN,...,Unspecified,Unspecified,NaN,NaN,3969250,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN


In [ ]:
#### Remove Data with mssing values 

In [37]:
label_col = 'NUMBER OF PERSONS INJURED'
cat_cols = ['CONTRIBUTING FACTOR VEHICLE 1', 'VEHICLE TYPE CODE 1']
lat_lon_cols = ['LATITUDE', 'LONGITUDE']
data = data_raw[input_cols +  lat_lon_cols + [label_col]].dropna()
data['CONTRIBUTING FACTOR VEHICLE 1'] = data['CONTRIBUTING FACTOR VEHICLE 1'].astype(str)
data['VEHICLE TYPE CODE 1'] = data['VEHICLE TYPE CODE 1'].astype(str)
data.describe()

,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED
count,1.382831e+06,1.382831e+06,1.382831e+06
mean,4.069448e+01,-7.387752e+01,2.612460e-01
std,1.095939e+00,2.282398e+00,6.573974e-01
min,0.000000e+00,-2.013600e+02,0.000000e+00
25%,4.066880e+01,-7.397737e+01,0.000000e+00
50%,4.072255e+01,-7.392990e+01,0.000000e+00
75%,4.076789e+01,-7.386698e+01,0.000000e+00
max,4.334444e+01,0.000000e+00,4.300000e+01


In [ ]:
#### Split Data 

In [38]:
df_train, df_test = train_test_split(data, test_size=0.25, random_state=42)
print('trainset shape {}'.format(df_train.shape))
print('testset shape {}'.format(df_test.shape))

trainset shape (1037123, 5)
testset shape (345708, 5)


,CONTRIBUTING FACTOR VEHICLE 1,VEHICLE TYPE CODE 1
0,Passing or Lane Usage Improper,Sedan
1,Pedestrian/Bicyclist/Other Pedestrian Error/Co...,Sedan
2,Failure to Yield Right-of-Way,Sedan
3,Driver Inattention/Distraction,Sedan
4,Unspecified,Sedan
...,...,...
105,Unspecified,Station Wagon/Sport Utility Vehicle
106,Driver Inattention/Distraction,Sedan
107,Failure to Yield Right-of-Way,Station Wagon/Sport Utility Vehicle
108,Unsafe Lane Changing,Station Wagon/Sport Utility Vehicle


In [70]:
from sklearn.feature_extraction.text import CountVectorizer
## COntributing Factors
min_df = .01
stop_words = ['and', 'the', 'as', 'that', 'but', 'in', 'back', 'know']

cv = CountVectorizer(min_df=min_df,  stop_words=stop_words,  ngram_range=(1, 3), binary=True)
# fits the count vect with a vocabuary 
cv = cv.fit(data['VEHICLE TYPE CODE 1'] )
cv_feature_names = cv.get_feature_names()
cv_feature_names

['bus',
 'com',
 'com veh',
 'other',
 'passenger',
 'passenger vehicle',
 'pick',
 'pick up',
 'pick up truck',
 'sedan',
 'sport',
 'sport utility',
 'sport utility station',
 'sport utility vehicle',
 'station',
 'station wagon',
 'station wagon sport',
 'taxi',
 'tires',
 'truck',
 'unknown',
 'up',
 'up truck',
 'utility',
 'utility station',
 'utility station wagon',
 'utility vehicle',
 'van',
 'veh',
 'vehicle',
 'wagon',
 'wagon sport',
 'wagon sport utility']

In [71]:
from sklearn.feature_extraction.text import CountVectorizer
## COntributing Factors
min_df = .01
stop_words = ['and', 'the', 'as', 'that', 'but', 'in', 'back', 'know']

cv2 = CountVectorizer(min_df=min_df,  stop_words=stop_words,  ngram_range=(2, 3), binary=True)
# fits the count vect with a vocabuary 
cv2 = cv2.fit(data['CONTRIBUTING FACTOR VEHICLE 1'] )
cv_feature_names2 = cv2.get_feature_names()
cv_feature_names2

['backing unsafely',
 'control disregarded',
 'driver inattention',
 'driver inattention distraction',
 'driver inexperience',
 'failure to',
 'failure to yield',
 'fatigued drowsy',
 'following too',
 'following too closely',
 'inattention distraction',
 'lane changing',
 'lane usage',
 'lane usage improper',
 'lost consciousness',
 'of way',
 'or lane',
 'or lane usage',
 'other vehicular',
 'passing or',
 'passing or lane',
 'passing too',
 'passing too closely',
 'right of',
 'right of way',
 'to yield',
 'to yield right',
 'too closely',
 'traffic control',
 'traffic control disregarded',
 'turning improperly',
 'unsafe lane',
 'unsafe lane changing',
 'usage improper',
 'yield right',
 'yield right of']

In [72]:
# define the number of clusters to use (number of centers )
n_centers = 10

# init the scaler normalizes values
scaler = StandardScaler()

# int the kmeans model
binner = KMeans(n_clusters=n_centers)

# package in a pipeline
lat_lon_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')),
                                      ('scaling', scaler), 
                                      ('binning and ohe', binner)])
lat_lon_transformer.fit(df_train[lat_lon_cols])

# this test thje transform and gets a bin number for each row, to plot
bins = np.argmin(lat_lon_transformer.transform(df_train[lat_lon_cols]), axis=1)

#example output form the lat lon transformer 
lat_lon_feature_names = list(range(lat_lon_transformer.steps[-1][1].n_clusters))
lat_lon_feature_names = ['Distance to loc:' + str(v) for v in lat_lon_feature_names]

# show an example of the transformation
pd.DataFrame(lat_lon_transformer.transform(df_train[lat_lon_cols].head(2)), columns=lat_lon_feature_names)



,Distance to loc:0,Distance to loc:1,Distance to loc:2,Distance to loc:3,Distance to loc:4,Distance to loc:5,Distance to loc:6,Distance to loc:7,Distance to loc:8,Distance to loc:9
0,0.096884,49.271386,56.125537,16.864445,0.156899,0.017390,0.167526,0.264184,0.108324,0.218156
1,0.071434,49.159997,56.106747,16.882079,0.009617,0.152273,0.070751,0.112399,0.085129,0.051911


In [84]:
feature_names =  cv.get_feature_names() + cv2.get_feature_names() + lat_lon_feature_names 
len(feature_names)


79

In [87]:
from scipy.sparse import hstack
def clean_data(df):
    f1 = cv.transform(df['VEHICLE TYPE CODE 1'] )
    f2 = cv2.transform(df['CONTRIBUTING FACTOR VEHICLE 1'] )
    f3 = lat_lon_transformer.transform(df[lat_lon_cols])
    return hstack((f1, f2, f3))

X_train  =  clean_data(df_train)
X_test =  clean_data(df_test)
y_train = df_train['NUMBER OF PERSONS INJURED'].values
y_test = df_test['NUMBER OF PERSONS INJURED'].values


In [90]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model =model.fit(X_train, y_train)
model.coef_

array([-0.16921973, -0.15786691, -0.04617715, -0.23693619, -0.036068  ,
       -0.036068  , -0.18711621,  0.05384231,  0.05384231, -0.16691881,
        0.03090146,  0.03090146,  0.00244661,  0.02845484,  0.03090146,
        0.03090146,  0.02845484, -0.12311856, -0.04617715, -0.2695473 ,
       -0.07237663,  0.05384231,  0.05384231,  0.03090146,  0.00244661,
        0.00244661,  0.02845484, -0.21526341, -0.04617715, -0.11885065,
       -0.3542823 ,  0.02845484,  0.02845484, -0.1600598 ,  0.02587419,
        0.00335409,  0.00335409, -0.02061119,  0.03770523,  0.01553507,
        0.00743886,  0.04652205,  0.04652205,  0.00335409, -0.03542205,
       -0.01847026, -0.01847026, -0.08984021,  0.01553507, -0.01847026,
       -0.01847026, -0.04288473, -0.01847026, -0.01847026, -0.08431471,
       -0.08431471,  0.01553507,  0.01553507,  0.01553507,  0.01553507,
       -0.03779265,  0.29014078,  0.02587419, -0.07608687, -0.03542205,
       -0.03542205, -0.01847026,  0.01553507,  0.01553507,  1.95

In [92]:
output = pd.DataFrame({'features':feature_names, 'coef':model.coef_}).sort_values('coef' , ascending=False)
output.head(10)

,features,coef
69,Distance to loc:0,1.956660
77,Distance to loc:8,0.830977
76,Distance to loc:7,0.493397
61,traffic control,0.290141
7,pick up,0.053842
8,pick up truck,0.053842
22,up truck,0.053842
21,up,0.053842
41,following too,0.046522
42,following too closely,0.046522


In [ ]:


# importas a color map from maplot lib
cmap = cm.get_cmap('Spectral')

# creates a scatter plot with bin number as coloring
df_train.plot.scatter(*lat_lon_cols, c=bins, cmap=cmap, s=5)
#plots the centers
for i in range(cluster_centers.shape[0]):
    plt.text(cluster_centers[i,0], cluster_centers[i,1], str(i))
plt.show()



